# Szyfrowanie w pełni holomorficzne - Laboratorium 02

## Zadanie 1.

Skopiuj kod pierścienia $\mathbb{Z}_{17}[X]/(X^4+1)$ wraz z arytmetyką oraz kod uproszczonej wersji algorytmu BGV z poprzednich zajęć - przydadzą się.

In [7]:
# pierścień Rq

import numpy as np

x4p1 = np.array([1, 0, 0, 0, 1])

class Z17:
    def __init__(self, poly: np.array):
        _, p = np.polydiv(poly, x4p1)
        self.poly = np.array([0,0,0,0])
        self.poly[-(p.shape[0]):] = p
        self.poly %= 17

    def __mul__(self, other):
        if isinstance(other, Z17):
            return Z17(np.polymul(self.poly, other.poly))
        return Z17(self.poly * other)

    __rmul__ = __mul__

    def __add__(self, other):
        return Z17(np.polyadd(self.poly, other.poly))

    __radd__ = __add__

    def __neg__(self):
        return -1 * self
    
    def __mod__(self, other):
        return Z17(self.poly % other)
    
    def __eq__(self,other):
        return self.poly == other.poly

    def __repr__(self):
        p =  [(f"{a}" if a != 1 else "") + f"x^{3 - i}" for i, a in enumerate(self.poly[:-1].astype(int)) if a != 0] 
        if self.poly[-1] != 0:
            p += [f"{int(self.poly[-1])}"]
        return " +".join(p)

In [8]:
# algorytm BGV

n = 4 
q = 17
t = 2

prob = 0.9

def SecretKeyGen(n):
    # sk =  np.random.binomial(2,0.5,size=n)-1
    sk = np.random.choice([-1,0,1],n,p=[0.,prob,1-prob])
    return Z17(sk)

def PubKeyGen(sk):
    a = Z17(np.random.randint(q,size=n))
    e = Z17(np.random.choice([-1,0,1],n,p=[0.,prob,1-prob]))
    
    pk0 = a*sk + t*e
    pk1 = -a
    return (pk0,pk1)

def Encrypt(m,pk):
    e0 = Z17(np.random.choice([-1,0,1],n,p=[0.,prob,1-prob]))
    e1 = Z17(np.random.choice([-1,0,1],n,p=[0.,prob,1-prob]))
    u = np.random.choice([-1,0,1],n,p=[0.,prob,1-prob])
    c0 = pk[0]*u + t*e0 + m
    c1 = pk[1]*u + t*e1
    return (c0,c1)

def Decrypt(c, sk, q, t):
    m = c[0] + c[1]*sk
    # m %= q # implicitly handled above
    m %= t
    return m

## Dodawanie `Add`

Załóżmy, że mamy dwie wiadomości zaszyfrowane tym samym kluczem prywatnym, tzn. dwie pary $(c_0,c_1)$ oraz $(c'_0,c'_1)$. Naturalnym sposobem zdefiniowania sumy jest $$c_0^{\ast}=c_0+c'_0$$ $$c_1^{\ast}=c_1+c'_1$$czyli szyfrogram sumy to $$c^{\ast}=(c_0^{\ast},c_1^{\ast}).$$

To podejście me jeden problem: z każdą operacją sumowania wzrasta zaszumienie końcowego szyfrogramu, co może skutkować błędnym deszyfrowaniem, jednak nie aż tak bardzo jak to się dzieje w przypadku mnożenia.

## Mnożenie `Mul`

Jak przy dodawaniu mamy dwie wiadomości zaszyfrowane tym samym kluczem prywatnym, tzn. dwie pary $(c_0,c_1)$ oraz $(c'_0,c'_1)$. W przypadku naturalnej definicji mnożenia sprawy się komplikują: jeżeli popatrzymy na funkcję `Decrypt`, to wiadomości $m$ i $m'$ kryjące się za naszymi szyfrogramami są postaci $$m=c_0+c_1s$$ $$m'=c'_0+c'_1s.$$Jeżeli teraz pomnożymy te dwie wiadomości, to otrzymamy $$mm'=(c_0+c_1s)(c'_0+c'_1s)=c_0c'_0+(c_0c'_1+c'_0c_1)s+c_1c'_1s^2$$
Otrzymujemy zatem **trzy** współrzędne końcowego szyfrogramu:
\begin{eqnarray*}
c^{\ast}_0=c_0c'_0\\
c^{\ast}_1=c_0c'_1+c'_0c_1\\
c^{\ast}_2=c_1c'_1
\end{eqnarray*}
Jako wynik mnożenia zwracamy szyfrogram $$c^{\ast}=(c_0^{\ast},c_1^{\ast},c_2^{\ast}).$$
W tym przypadku oprócz problemu z narastającym zaszumieniem mamy jeszcze problem z dodatkową współrzędną, której nie bierze pod uwagę nasza implementacja funkcji deszyfrującej.

## Prosta relinearyzacja `KeySwitch`

Niech $c^{\ast}=(c_0^{\ast},c_1^{\ast},c_2^{\ast})$ będzie wynikiem mnożenia dwóch wiadomości $m_1$ i $m_2$ zaszyfrowanych przy pomocy klucza publicznego $(pk_0, pk_1)$ i klucza prywatnego $s$. Żeby pozbyć się współrzędnej $c_2^{\ast}$ (i przekształcić postać iloczynu $mm'$ z kwadratowej na liniową) stosujemy *zmianę klucza*.

**Krok 1 - rozkład wielomianu.** Najpierw zapisujemy wszystkie współczynniki wielomianu $c_2^{\ast}=w_0+w_1x+w_2x^2+w_3x^3$ w reprezentacji w systemie dwójkowym, tzn. $$w_i=\sum_{j=0}^{\lfloor \log_2 q\rfloor+1}2^jw^{(j)}_i,\ \ i=0,1,2,3$$

Konstruujemy nowe wielomiany dla $j=0,...,\lfloor \log_2 q\rfloor+1$ $$c_2^{\ast (j)}=w^{(j)}_0+w^{(j)}_1x+w^{(j)}_2x^2+w^{(j)}_3x^3$$i za ich pomocą rozkładamy wielomian $c_2^{\ast}$ $$c_2^{\ast}=\sum_{j=0}^{\lfloor \log_2 q\rfloor+1}2^j c_2^{\ast (j)}\mod q$$

**Krok 2 - generowanie wskazówek.** Dla $j=0,...,\lfloor \log_2 q\rfloor+1$ z klucza prywatengo $s$ generujemy tzw. *wskazówki*: $$(ek_0^{(j)},ek_1^{(j)})=(a_js+te_j+2^js^2,-a_j),$$gdzie $a_j\in R_q$ są generowane losowo z rozkładu jednostajnego a błedy $e_i\in R_q$ - losowo z rozkładu typu Gaussowskiego (jak przy generowaniu kluczy).

**Krok 3 - nowy szyfrogram.** Generujemy nowy szyfrogram $(\widehat{c}_0,\widehat{c}_1)$: $$\widehat{c}_0=c_0^{\ast}+\sum_{j=0}^{\lfloor \log_2 q\rfloor+1}ek_0^{(j)}c_2^{\ast (j)}$$
$$\widehat{c}_1=c_1^{\ast}+\sum_{j=0}^{\lfloor \log_2 q\rfloor+1}ek_1^{(j)}c_2^{\ast (j)}$$

Po zdeszyfrowaniu $(\widehat{c}_0,\widehat{c}_1)$ z kluczem $s$ powinniśmy otrzymać wiadomość będącą wynikiem mnożenia dwóch wiadomości $m_1$ i $m_2$.

## Zadanie 2.

Zaimplementuj funkcje `Add`, `KeySwitch` oraz `Mul` realizujące powyższe algorytmy.
- sprawdź działanie dodawania i mnożenia dla jednej operacji ($m_1+m_2, m_1*m_2$). Pamiętaj o wykorzystaniu funkcji `KeySwitch` przy mnożeniu. Dobierz parametry kryptosystemu tak, żeby po deszyfrowaniu otrzymać poprawne wyniki.
- ile operacji dodawania możemy wykonać zanim narastające błędy spowodują błędne deszyfrowanie ($m_1+m_2+m_3+...$)?
- a ile operacji mnożenia ($m_1*m_2*m_3*...$)?
- sprawdź jak wygląda skuteczność deszyfrowania w przypadku mieszania operacji, np. $m_1*m_2+m_3$. Dla jakiej głębokości $N$ operacji mieszanych na wiadomościach deszyfrowanie jest poprawne?

Przez głębokość $N$ operacji mieszanych rozumiemy kombinację postaci: iloczyn $N$ wiadomości plus iloczyn $N-1$ wiadomości plus iloczyn $N-2$ wiadomości plus ... plus iloczyn dwóch wiadomości plus jedna wiadomość.

Otrzymane rezultaty (maksymalna głębokość operacji dodawania, operacji mnożenia i operacji mieszanych) opisz pełnym zdaniem w osobnej komórce pod testami.

In [13]:
# Add
def add_z17(c_x,c_y):
    return (c_x[0]+c_y[0],c_x[1]+c_y[1])


# KeySwitch
pow_max = np.floor(np.log2(q)).astype(int)+1

def key_switch(c_x,c_y,c_z):
    W = np.zeros(shape = (c_z.poly.shape[0],pow_max))
    
    for i, w in enumerate(c_z.poly):
        for j, wj in enumerate(bin(w)[-1:1:-1]):
            W[i][j] = int(wj)
    
    C = [Z17(p[::-1]) for p in W.T]
    
    a = Z17(np.random.randint(q,size=n))
    e = Z17(np.random.choice([-1,0,1],n,p=[0.,prob,1-prob]))
    
   
        
    
    


# Mul

def mul_z17(c_x,c_y):
    pass
    


In [16]:
c_z = Z17(np.random.randint(q,size=n))
kk = key_switch(None,None,c_z)

### Opis rezultatów - TODO

